In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/API_PA.NUS.PRVT.PP_DS2_en_csv_v2_4157455.csv", skiprows=3)

In [3]:
df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       'Unnamed: 66'],
      dtype='object')

In [4]:
df = df[["Country Name", "Country Code", "2017", "2018", "2019"]]
df = df.rename(columns={"Country Name": "Country"})
df.head()

,Country,Country Code,2017,2018,2019
0,Aruba,ABW,1.480481,1.497584,1.533552
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN
2,Afghanistan,AFG,17.360185,17.052368,17.134464
3,Africa Western and Central,AFW,NaN,NaN,NaN
4,Angola,AGO,107.269989,125.265871,144.052274


In [5]:
df["2019"] = df["2019"].fillna(df["2018"])
df["2019"] = df["2019"].fillna(df["2017"])

In [6]:
df = df.sort_values(by="Country").reset_index(drop=True)
df = df.drop(columns=["2017", "2018"])
df = df.dropna()

In [7]:
curr = pd.read_csv("countries.csv")

In [8]:
curr.columns

Index(['name', 'name_fr', 'ISO3166-1-Alpha-2', 'ISO3166-1-Alpha-3',
       'ISO3166-1-numeric', 'ITU', 'MARC', 'WMO', 'DS', 'Dial', 'FIFA', 'FIPS',
       'GAUL', 'IOC', 'currency_alphabetic_code', 'currency_country_name',
       'currency_minor_unit', 'currency_name', 'currency_numeric_code',
       'is_independent'],
      dtype='object')

In [9]:
curr = curr[["name", "ITU", "currency_alphabetic_code", "currency_name"]]
curr = curr.rename(columns={"name":"Country", "currency_alphabetic_code":"Currency", "currency_name": "Currency Name"})

In [10]:
s1 = pd.merge(df, curr, on=["Country"])

In [11]:
final = pd.merge(df, s1, on=["Country"])

In [12]:
final.columns = final.columns.str.replace(r'_x$', '')
final.columns = final.columns.str.replace(r'_y$', '')
final = final.loc[:, ~final.columns.duplicated()]

C:\Users\prabo\AppData\Local\Temp\ipykernel_18160\3792912422.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final.columns = final.columns.str.replace(r'_x$', '')
C:\Users\prabo\AppData\Local\Temp\ipykernel_18160\3792912422.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  final.columns = final.columns.str.replace(r'_y$', '')


In [13]:
final.head()

,Country,Country Code,2019,ITU,Currency,Currency Name
0,Afghanistan,AFG,17.134464,AFG,AFN,Afghani
1,Albania,ALB,53.392465,ALB,ALL,Lek
2,Algeria,DZA,38.944652,ALG,DZD,Algerian Dinar
3,Angola,AGO,144.052274,AGL,AOA,Kwanza
4,Antigua and Barbuda,ATG,2.429231,ATG,XCD,East Caribbean Dollar


In [14]:
final = final.drop(columns=["ITU"])
final = final[["Country", "Currency", "Currency Name", "2019"]]
final.head()

,Country,Currency,Currency Name,2019
0,Afghanistan,AFN,Afghani,17.134464
1,Albania,ALL,Lek,53.392465
2,Algeria,DZD,Algerian Dinar,38.944652
3,Angola,AOA,Kwanza,144.052274
4,Antigua and Barbuda,XCD,East Caribbean Dollar,2.429231


In [15]:
final.shape

(163, 4)

In [16]:
def convert_by_ppp(source_country, target_country, salary):
    return salary / list(final[final["Country"]==source_country]["2019"])[0] * list(final[final["Country"]==target_country]["2019"])[0]

In [17]:
val = convert_by_ppp("United States", "Canada", 150000)
round(val, 2)

199696.5

In [18]:
final.to_csv("data.csv", index=False)